# Preamble

In [1]:
import os, sys, glob
import pandas as pd
import numpy as np
import scipy.io as sio

In [2]:
sys.path.append('/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/code/func/')
from proj_environment import set_proj_env
from func import node_strength, ave_control, modal_control

In [3]:
train_test_str = 'squeakycleanExclude'
exclude_str = 't1Exclude' # 't1Exclude' 'fsFinalExclude'
parc_str = 'schaefer' # 'schaefer' 'lausanne'
parc_scale = 400 # 200 400 | 60 125
parcel_names, parcel_loc, drop_parcels, num_parcels, yeo_idx, yeo_labels = set_proj_env(train_test_str = train_test_str, exclude_str = exclude_str,
                                                                            parc_str = parc_str, parc_scale = parc_scale)

### Setup output directory

In [4]:
print(os.environ['MODELDIR'])
if not os.path.exists(os.environ['MODELDIR']): os.makedirs(os.environ['MODELDIR'])

/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400


## Load train/test .csv and setup node .csv

In [5]:
os.path.join(os.environ['TRTEDIR'])

'/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude'

In [6]:
# Load data
df = pd.read_csv(os.path.join(os.environ['TRTEDIR'], 'df_pheno.csv'))
df.set_index(['bblid', 'scanid'], inplace = True)
print(df.shape)

(1100, 18)


In [7]:
# Missing data file for this subject only for schaefer 200
if parc_str == 'schaefer' and parc_scale == 200:
    df.drop(labels = (112598, 5161), inplace=True)

In [8]:
# output dataframe
ct_labels = ['ct_' + str(i) for i in range(num_parcels)]
gmd_labels = ['gmd_' + str(i) for i in range(num_parcels)]

df_node = pd.DataFrame(index = df.index, columns = ct_labels + gmd_labels)
df_node.insert(0, train_test_str, df[train_test_str])

print(df_node.shape)

(1100, 801)


## Load in cortical thickness

In [9]:
CT = np.zeros((df.shape[0], num_parcels))

for (i, (index, row)) in enumerate(df.iterrows()):
    file_name = os.environ['CT_NAME_TMP'].replace("scanid", str(index[1]))
    full_path = os.path.join(os.environ['CTDIR'], file_name)
    
    ct = np.loadtxt(full_path)
    CT[i,:] = ct
    
df_node.loc[:,ct_labels] = CT

## Load in gray matter density

In [10]:
GMD = np.zeros((df.shape[0], num_parcels))

for (i, (index, row)) in enumerate(df.iterrows()):
    file_name = os.environ['GMD_NAME_TMP'].replace("scanid", str(index[1]))
    full_path = os.path.join(os.environ['GMDDIR'], file_name)
    
    gmd = np.loadtxt(full_path)
    GMD[i,:] = gmd
    
df_node.loc[:,gmd_labels] = GMD

## Save out

In [11]:
os.environ['MODELDIR']

'/Users/lindenmp/Dropbox/Work/ResProjects/NormativeNeuroDev_CrossSec_T1/analysis/normative/t1Exclude/squeakycleanExclude/schaefer_400'

In [12]:
# Save out
df_node.to_csv(os.path.join(os.environ['MODELDIR'], 'df_node_base.csv'))
df.to_csv(os.path.join(os.environ['MODELDIR'], 'df_pheno.csv'))